In [1]:
import pandas as pd
import numpy as np
import torch

# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
%autoreload 2

# Basic import
import eugene as eu
eu.__version__

Global seed set to 13


'0.0.6'

# Test data

In [2]:
sdata = eu.datasets.random1000()

In [3]:
eu.pp.ohe_seqs_sdata(sdata)

One-hot encoding sequences:   0%|          | 0/1000 [00:00<?, ?it/s]

SeqData object modified:
	ohe_seqs: None -> 1000 ohe_seqs added


In [4]:
eu.pp.reverse_complement_seqs_sdata(sdata)

SeqData object modified:
	ohe_rev_seqs: None -> 1000 ohe_rev_seqs added


# Reference registry

In [53]:
from typing import Union, Callable

def ablate_first_base(seqs):
    """
    Change the first base of each sequence in `seqs` to A, C, G, or T.
    This is used for computing the DeepLIFT scores for the first base
    """
    seqs[:, :, 0] = [0, 0, 0, 0]
    return seqs

def zero_ref_seqs(seqs):
    seqs = torch.zeros(seqs.shape)
    return seqs

def gc_ref_seqs(seqs, dists=None):
    if dists is None:
        dists = torch.Tensor([0.3, 0.2, 0.2, 0.3])
        seqs = dists.expand(seqs.shape[0], seqs.shape[2], 4).transpose(2, 1)
    else:
        seqs = dists.expand(seqs.shape[0], seqs.shape[2], 4).transpose(2, 1)
    return seqs

REFERENCE_REGISTRY = {
    "zero": zero_ref_seqs,
    "gc": gc_ref_seqs,
    "shuffle": dinuc_shuffle_seqs,
}

def _get_reference(
    seqs: np.ndarray,
    method: Union[str, Callable],
):
    """
    Returns torch.Tensor reference
    """
    if isinstance(method, str):
        if method not in REFERENCE_REGISTRY:
            raise ValueError(f"Reference method {method} not in {list(REFERENCE_REGISTRY.keys())}")
        if isinstance(seqs, tuple):
            return tuple([torch.Tensor(REFERENCE_REGISTRY[method](seqs[i])) for i in range(len(seqs))])
        else:
            return torch.tensor(REFERENCE_REGISTRY[method](seqs))
    elif callable(method):
        if isinstance(seqs, tuple):
            return tuple([torch.Tensor(method(seqs[i])) for i in range(len(seqs))])
        else:
            return torch.Tensor(method(seqs))
    else:
        raise ValueError(f"Reference method {method} not in {list(REFERENCE_REGISTRY.keys())}")

In [55]:
refs = _get_reference(ohe_seqs, "shuffle")

In [57]:
refs = _get_reference((ohe_seqs, ohe_seqs), "shuffle")

---